In [1]:
import nltk

## 9.1 语法特征(也叫文法特征)

在基于规则的上下文语法中，特征——值偶对被称为特征结构。

In [2]:
# 字典存储特征以及特征的值

# -   'CAT' 表示语法类别；'ORTH'：表示正字法（正词法，拼写规则）
# -   'REF' 表示 'kim' 的指示物；'REL'：表示 'chase' 表示的关系
# -   'AGT' 表示施事（agent）角色；'PAT'： 表示受事（patient）角色
kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
chase = {'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase'}
chase['AGT'] = 'sbj'  # 'sbj'（主语）作为占位符
chase['PAT'] = 'obj'  # 'obj'（宾语）作为占位符
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}

In [3]:
sent = "Kim chased Lee"
tokens = sent.split()
tokens

['Kim', 'chased', 'Lee']

In [4]:
def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            return fs

In [5]:
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])
subj,verb,obj

({'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'},
 {'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase', 'AGT': 'sbj', 'PAT': 'obj'},
 {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'})

In [6]:
verb['AGT'] = subj['REF']  # agent of 'chase' is Kim
verb['PAT'] = obj['REF']  # patient of 'chase' is Lee
for k in ['ORTH', 'REL', 'AGT', 'PAT']:  # check featstruct of 'chase'
    print("%-5s => %s" % (k, verb[k]))

ORTH  => chased
REL   => chase
AGT   => k
PAT   => l


In [7]:
# 'SRC'：表示源事（source）的角色；'EXP'：表示体验者（experiencer）的角色
surprise = {'CAT': 'V', 'ORTH': 'surprised', 'REL': 'surprise', 'SRC': 'SBJ', 'EXP': 'obj'}
surprise
# 特征结构是非常强大的，特征的额外表现力（Ref：Sec 9.3）开辟了用于描述语言结构复杂性的可能。

{'CAT': 'V',
 'ORTH': 'surprised',
 'REL': 'surprise',
 'SRC': 'SBJ',
 'EXP': 'obj'}

### 9.1.1 句法协议

协议（agreement）：动词的形态属性和主语名词短语的句法属性一起变化的过程。

表9-1 英语规则动词的协议范式


### 9.1.2 使用属性和约束

In [8]:
# Ex9-1 基于特征的语法的例子
nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'


In [9]:
# Ex9-2 跳跃基于特征的图表分析器
tokens = 'Kim likes children'.split()
from nltk import load_parser

cp = load_parser('grammars/book_grammars/feat0.fcfg', trace = 2)
for tree in cp.parse(tokens):
    print(tree)

|.Kim .like.chil.|
Leaf Init Rule:
|[----]    .    .| [0:1] 'Kim'
|.    [----]    .| [1:2] 'likes'
|.    .    [----]| [2:3] 'children'
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] PropN[NUM='sg'] -> 'Kim' *
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] NP[NUM='sg'] -> PropN[NUM='sg'] *
Feature Bottom Up Predict Combine Rule:
|[---->    .    .| [0:1] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'sg'}
Feature Bottom Up Predict Combine Rule:
|.    [----]    .| [1:2] TV[NUM='sg', TENSE='pres'] -> 'likes' *
Feature Bottom Up Predict Combine Rule:
|.    [---->    .| [1:2] VP[NUM=?n, TENSE=?t] -> TV[NUM=?n, TENSE=?t] * NP[] {?n: 'sg', ?t: 'pres'}
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] N[NUM='pl'] -> 'children' *
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] NP[NUM='pl'] -> N[NUM='pl'] *
Feature Bottom Up Predict Combine Rule:
|.    .    [---->| [2:3] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'pl'}
Feature Single Edge Fundame

### 9.1.3 术语

简单的值通常称为原子。

-   原子值的一种特殊情况是布尔值。

AGR是一个复杂值。

属性——值矩阵（Attribute-Value Matrix，AVM）


## 9.2 处理特征结构

In [10]:
# 特征结构的构建；两个不同特征结构的统一（合一）运算。
fs1 = nltk.FeatStruct(TENSE = 'past', NUM = 'sg')
fs1

[NUM='sg', TENSE='past']

In [11]:
fs1 = nltk.FeatStruct(PER = 3, NUM = 'pl', GND = 'fem')
print("fs1['GND']=",fs1['GND'])
fs1['CASE'] = 'acc'
show_subtitle("fs1")
fs1

fs1['GND']= fem
--------------- >fs1< ---------------


[CASE='acc', GND='fem', NUM='pl', PER=3]

In [12]:
fs2 = nltk.FeatStruct(POS = 'N', AGR = fs1)
show_subtitle("fs2['AGR']")
fs2['AGR']

--------------- >fs2['AGR']< ---------------


[CASE='acc', GND='fem', NUM='pl', PER=3]

In [13]:
show_subtitle("fs2")
fs2

--------------- >fs2< ---------------


[AGR=[CASE='acc', GND='fem', NUM='pl', PER=3], POS='N']

In [14]:
nltk.FeatStruct("[POS='N',AGR=[PER=3, NUM='pl', GND='fem']]")

[AGR=[GND='fem', NUM='pl', PER=3], POS='N']

In [15]:
# 特征结构也可以用来表示其他数据
nltk.FeatStruct(NAME = 'Lee', TELNO = '13918181818', AGE = 33)

[AGE=33, NAME='Lee', TELNO='13918181818']

特征结构也可以使用有向无环图（Directed Acyclic Graph，DAG）来表示，相当于前面的AVM

DAG可以使用结构共享或者重入来表示两条路径具有相同的值，即它们是等价的。

In [16]:
# 括号()里面的整数称为标记或者同指标志（coindex）。
nltk.FeatStruct("""[Name='Lee', ADDRESS=(1)[NUMBER=74,STREET='rue Pascal'],SPOUSE=[NAME='Kim',ADDRESS->(1)]]""")


[ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'], Name='Lee', SPOUSE=[ADDRESS->(1), NAME='Kim']]

### 9.2.1 包含（蕴涵） 和 统一（合一） （Ref：《自然语言处理综论》Ch15）

一般的特征结构包含（蕴涵）特殊的特征结构

合并两个特征结构的信息称为统一（合一），统一（合一）运算是对称的。

In [17]:
# 合一的相容运算
fs1 = nltk.FeatStruct(NUMBER = 74, STREET = 'rule Pascal')
fs2 = nltk.FeatStruct(CITY = 'Paris')
show_subtitle("fs1.unify(fs2)")
print(fs1.unify(fs2))
show_subtitle("fs2.unify(fs1)")
print(fs2.unify(fs1))
show_subtitle("fs1.unify(fs2) == fs2.unify(fs1)")
print(fs1.unify(fs2) == fs2.unify(fs1))

--------------- >fs1.unify(fs2)< ---------------
[ CITY   = 'Paris'       ]
[ NUMBER = 74            ]
[ STREET = 'rule Pascal' ]
--------------- >fs2.unify(fs1)< ---------------
[ CITY   = 'Paris'       ]
[ NUMBER = 74            ]
[ STREET = 'rule Pascal' ]
--------------- >fs1.unify(fs2) == fs2.unify(fs1)< ---------------
True


In [18]:
# 合一的失败运算
fs0 = nltk.FeatStruct(A = 'a')
fs1 = nltk.FeatStruct(A = 'b')
fs2 = fs0.unify(fs1)
print(fs2)

None


In [19]:
fs0 = nltk.FeatStruct("[SPOUSE=[ADDRESS=[CITY=Paris]]]")
# 无同指标志的特征结构的相容运算
fs1 = nltk.FeatStruct("""[Name='Lee', 
ADDRESS=[NUMBER=74,STREET='rue Pascal'],
SPOUSE=[NAME='Kim',
ADDRESS=[NUMBER=74,STREET='rue Pascal']]]""")
print(fs0.unify(fs1))

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ Name    = 'Lee'                                   ]
[                                                   ]
[           [           [ CITY   = 'Paris'      ] ] ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]


In [20]:
# 有同指标志的特征结构的相容运算
fs2 = nltk.FeatStruct("""[Name='Lee', 
ADDRESS=(1)[NUMBER=74,STREET='rue Pascal'],
SPOUSE=[NAME='Kim',
ADDRESS->(1)]]""")
print(fs0.unify(fs2))

[               [ CITY   = 'Paris'      ] ]
[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ Name    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


In [21]:
# 使用变量?x表示的特征结构的相容运算
fs1 = nltk.FeatStruct("[ADDRESS1=[NUMBER=74, STREET='rue Pascal'], ADDRESS4=[NAME='Lee']]")
fs2 = nltk.FeatStruct("[ADDRESS1=?x, ADDRESS2=?x, ADDRESS3=?y, ADDRESS4=?y]")
show_subtitle("fs2")
print(fs2)
show_subtitle("fs2.unify(fs1)")
print(fs2.unify(fs1))

--------------- >fs2< ---------------
[ ADDRESS1 = ?x ]
[ ADDRESS2 = ?x ]
[ ADDRESS3 = ?y ]
[ ADDRESS4 = ?y ]
--------------- >fs2.unify(fs1)< ---------------
[ ADDRESS1 = (1) [ NUMBER = 74           ] ]
[                [ STREET = 'rue Pascal' ] ]
[                                          ]
[ ADDRESS2 -> (1)                          ]
[                                          ]
[ ADDRESS3 = (2) [ NAME = 'Lee' ]          ]
[                                          ]
[ ADDRESS4 -> (2)                          ]


## 9.3 扩展基于特征的文法（语法）
### 9.3.1 子类别（次范畴化）

广义短语结构语法（Generalized Phrase Structure Grammar，GPSG），

允许词汇类别支持SUBCAT特征（表明项目所属的子类别）

### 9.3.2 回顾核心词概念

### 9.3.3 助动词和倒装


### 9.3.4 无限制依赖成分

In [22]:
# 具有倒装从句和长距离依赖的产生式的语法，使用斜线类别
nltk.data.show_cfg('grammars/book_grammars/feat1.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
S[-INV] -> NP VP
S[-INV]/?x -> NP VP/?x
S[-INV] -> NP S/NP
S[-INV] -> Adv[+NEG] S[+INV]
S[+INV] -> V[+AUX] NP VP
S[+INV]/?x -> V[+AUX] NP VP/?x
SBar -> Comp S[-INV]
SBar/?x -> Comp S[-INV]/?x
VP -> V[SUBCAT=intrans, -AUX]
VP -> V[SUBCAT=trans, -AUX] NP
VP/?x -> V[SUBCAT=trans, -AUX] NP/?x
VP -> V[SUBCAT=clause, -AUX] SBar
VP/?x -> V[SUBCAT=clause, -AUX] SBar/?x
VP -> V[+AUX] VP
VP/?x -> V[+AUX] VP/?x
# ###################
# Lexical Productions
# ###################
V[SUBCAT=intrans, -AUX] -> 'walk' | 'sing'
V[SUBCAT=trans, -AUX] -> 'see' | 'like'
V[SUBCAT=clause, -AUX] -> 'say' | 'claim'
V[+AUX] -> 'do' | 'can'
NP[-WH] -> 'you' | 'cats'
NP[+WH] -> 'who'
Adv[+NEG] -> 'rarely' | 'never'
NP/NP ->
Comp -> 'that'


In [23]:
tokens = 'who do you claim that you like'.split()
from nltk import load_parser

cp = load_parser('grammars/book_grammars/feat1.fcfg')
tree=[]
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (NP[+WH] who)
  (S[+INV]/NP[]
    (V[+AUX] do)
    (NP[-WH] you)
    (VP[]/NP[]
      (V[-AUX, SUBCAT='clause'] claim)
      (SBar[]/NP[]
        (Comp[] that)
        (S[-INV]/NP[]
          (NP[-WH] you)
          (VP[]/NP[] (V[-AUX, SUBCAT='trans'] like) (NP[]/NP[] )))))))


In [24]:
tokens = 'you claim that you like cats'.split()
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (NP[-WH] you)
  (VP[]
    (V[-AUX, SUBCAT='clause'] claim)
    (SBar[]
      (Comp[] that)
      (S[-INV]
        (NP[-WH] you)
        (VP[] (V[-AUX, SUBCAT='trans'] like) (NP[-WH] cats))))))


In [25]:
tokens = 'rarely do you sing'.split()
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (Adv[+NEG] rarely)
  (S[+INV]
    (V[+AUX] do)
    (NP[-WH] you)
    (VP[] (V[-AUX, SUBCAT='intrans'] sing))))


### 9.3.5 德语中的格和性别

In [26]:
# Ex9-4 基于特征的语法的例子（表示带格的协议的相互作用）
nltk.data.show_cfg('grammars/book_grammars/german.fcfg')

% start S
# Grammar Productions
S -> NP[CASE=nom, AGR=?a] VP[AGR=?a]
NP[CASE=?c, AGR=?a] -> PRO[CASE=?c, AGR=?a]
NP[CASE=?c, AGR=?a] -> Det[CASE=?c, AGR=?a] N[CASE=?c, AGR=?a]
VP[AGR=?a] -> IV[AGR=?a]
VP[AGR=?a] -> TV[OBJCASE=?c, AGR=?a] NP[CASE=?c]
# Lexical Productions
# Singular determiners
# masc
Det[CASE=nom, AGR=[GND=masc,PER=3,NUM=sg]] -> 'der' 
Det[CASE=dat, AGR=[GND=masc,PER=3,NUM=sg]] -> 'dem'
Det[CASE=acc, AGR=[GND=masc,PER=3,NUM=sg]] -> 'den'
# fem
Det[CASE=nom, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die' 
Det[CASE=dat, AGR=[GND=fem,PER=3,NUM=sg]] -> 'der'
Det[CASE=acc, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die' 
# Plural determiners
Det[CASE=nom, AGR=[PER=3,NUM=pl]] -> 'die' 
Det[CASE=dat, AGR=[PER=3,NUM=pl]] -> 'den' 
Det[CASE=acc, AGR=[PER=3,NUM=pl]] -> 'die' 
# Nouns
N[AGR=[GND=masc,PER=3,NUM=sg]] -> 'Hund'
N[CASE=nom, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[CASE=dat, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunden'
N[CASE=acc, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[AGR=[GND=fem,PER=3,

In [27]:
tokens = 'ich folge den Katzen'.split()
cp = load_parser('grammars/book_grammars/german.fcfg')
for tree in cp.parse(tokens):
    print(tree)

(S[]
  (NP[AGR=[NUM='sg', PER=1], CASE='nom']
    (PRO[AGR=[NUM='sg', PER=1], CASE='nom'] ich))
  (VP[AGR=[NUM='sg', PER=1]]
    (TV[AGR=[NUM='sg', PER=1], OBJCASE='dat'] folge)
    (NP[AGR=[GND='fem', NUM='pl', PER=3], CASE='dat']
      (Det[AGR=[NUM='pl', PER=3], CASE='dat'] den)
      (N[AGR=[GND='fem', NUM='pl', PER=3]] Katzen))))


In [28]:
tokens='ich folge den Katze'.split()
cp = load_parser('grammars/book_grammars/german.fcfg',trace=2)
for tree in cp.parse(tokens):
    print(tree)

|.ich.fol.den.Kat.|
Leaf Init Rule:
|[---]   .   .   .| [0:1] 'ich'
|.   [---]   .   .| [1:2] 'folge'
|.   .   [---]   .| [2:3] 'den'
|.   .   .   [---]| [3:4] 'Katze'
Feature Bottom Up Predict Combine Rule:
|[---]   .   .   .| [0:1] PRO[AGR=[NUM='sg', PER=1], CASE='nom'] -> 'ich' *
Feature Bottom Up Predict Combine Rule:
|[---]   .   .   .| [0:1] NP[AGR=[NUM='sg', PER=1], CASE='nom'] -> PRO[AGR=[NUM='sg', PER=1], CASE='nom'] *
Feature Bottom Up Predict Combine Rule:
|[--->   .   .   .| [0:1] S[] -> NP[AGR=?a, CASE='nom'] * VP[AGR=?a] {?a: [NUM='sg', PER=1]}
Feature Bottom Up Predict Combine Rule:
|.   [---]   .   .| [1:2] TV[AGR=[NUM='sg', PER=1], OBJCASE='dat'] -> 'folge' *
Feature Bottom Up Predict Combine Rule:
|.   [--->   .   .| [1:2] VP[AGR=?a] -> TV[AGR=?a, OBJCASE=?c] * NP[CASE=?c] {?a: [NUM='sg', PER=1], ?c: 'dat'}
Feature Bottom Up Predict Combine Rule:
|.   .   [---]   .| [2:3] Det[AGR=[GND='masc', NUM='sg', PER=3], CASE='acc'] -> 'den' *
|.   .   [---]   .| [2:3] Det[AGR=[

## 9.4 小结
*   上下文无关语法的传统分类是原子符号。特征结构的重要作用之一是捕捉精细的区分，否则将需要数量翻倍的原子类别
*   通过使用特征值的变量，可以表达出语法产生式中的限制，使得不同的特征规格之间相互依赖
*   在词汇层面指定固定的特征值，并且限制短语中的特征值，使其与“孩子”的对应值相统一（？）
*   特征值可以是原子的，也可以是复杂的。原子值的特定类别是布尔值
*   两个特征可以共享一个值（原子的或者复杂的）。具有共享值的结构被称为重入。共享值被表示AVM中的数字索引（或者标记）
*   特征结构中的路径是特征元组，对应着从图底部开始的弧序列上的标签。
*   如果两条路径共享一个值，那么这两条路径是等价的。
*   包含的特征结构是偏序的。特征结构A蕴涵特征结构B，说明特征结构A更加一般，特征结构B更加特征
*   如果统一（合一）运算在特征结构中指定了一条路径，那么同时指定了所有与这条路径等价的其他路径。
*   使用特征结构对大量的语言学现象进行简洁的分析，包括：
    *   动词子类别
    *   倒装结构
    *   无限制依赖结构
    *   格支配

## 9.5 深入阅读

理论语言学中使用特征是捕捉语音的音素特征

计算语言学提出了语言功能可以被属性——值结构统一捕获

词汇功能语法表示语法关系和与成分结构短语关联的谓词参数结构